In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
# !pip install unsloth_zoo unsloth vllm
!pip install -U bitsandbytes datasets

In [3]:
# from unsloth import FastModel
# from unsloth.chat_templates import get_chat_template
# model, tokenizer = FastModel.from_pretrained(
#     model_name = "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     max_seq_length = 2048, # Choose any for long context!
#     load_in_4bit = True,  # 4 bit quantization to reduce memory
#     load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
#     full_finetuning = False, # [NEW!] We have full finetuning now!
#     # token = "hf_...", # use one if using gated models
# )

In [5]:
#@title for non unsloth models
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from peft import PeftModel
import torch

# base
model_id = "aisingapore/Llama-SEA-LION-v3.5-8B-R"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

# fine-tuned
model_base = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
peft_model_id = "ShoAnn/Llama-SEA-LION-v3.5-8B-R-legalqa"
model = PeftModel.from_pretrained(model_base, peft_model_id)
model.merge_and_unload()

model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


# **Load data**

In [6]:
#@title Load ground truth

In [7]:
# from unsloth.chat_templates import get_chat_template
# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "gemma-3",
#     # enable_thinking = False
# )

In [ ]:
from datasets import DatasetDict, load_dataset
hub_dataset = load_dataset("ShoAnn/legalqa_klinik_hukumonline")
test = hub_dataset["test"]

# Gather the splits into a DatasetDict
dataset = DatasetDict({
    "test": test,
})

instruction_prompt = "You are an AI Legal Assistant. Your task is to carefully analyze the provided **Legal Basis** below and answer given **Question** based *solely* and *exclusively* on the information contained within that context in Indonesian Language."
def format_to_conv(examples):
    questions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answer"]
    conversation = []
    for question, input, output in zip(questions, inputs, outputs):
        conversation.append([ # This line has been changed
            {'content': f'{instruction_prompt} \n **Question** \n{question} \n**Legal Basis** \n{input}', 'role': 'user'},
        ])
    return { "conversations" : conversation, }

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = True, thinking_mode="off") for convo in convos]
   return { "text" : texts, }

conv_test = dataset['test'].map(format_to_conv, batched = True,)
test = conv_test.map(formatting_prompts_func, batched = True)

README.md:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

In [ ]:
test['text'][0]

In [ ]:
#@title Load inference result from my model

In [ ]:
import os
import json
from tqdm import tqdm
import pandas as pd
import torch

def save_checkpoint(inference_results, checkpoint_file):
    """Save the current state of inference"""
    inference_results.to_csv(checkpoint_file, index=False)

def load_checkpoint(checkpoint_file):
    """Load the previous state of inference"""
    if os.path.exists(checkpoint_file):
        return pd.read_csv(checkpoint_file)
    return pd.DataFrame()

def run_inference_with_checkpointing(dataset, model, tokenizer, checkpoint_file='checkpoint.csv',
                                   save_frequency=100):
    """
    Run inference with periodic checkpointing and text streaming

    Args:
        dataset: The input dataset
        model: The model to use for inference
        tokenizer: The tokenizer to use
        checkpoint_file: Path to save/load checkpoint
        save_frequency: How often to save checkpoints (in number of rows)
    """
    # Load checkpoint if it exists
    inference_results = load_checkpoint(checkpoint_file)

    # Find where to resume from
    processed_texts = set(inference_results['text'].tolist() if not inference_results.empty else [])
    remaining_rows = [row for row in dataset if row['text'] not in processed_texts]

    print(f"Resuming from {len(processed_texts)} previously processed items")

    # Create a text streamer for real-time output
    # streamer = TextStreamer(tokenizer, skip_special_tokens=True)

    try:
        for i, row in enumerate(tqdm(remaining_rows)):

            # llama 3.1 & cendol inference setup
            # inputs = tokenizer([row['text']], return_tensors="pt").to("cuda")
            # outputs = model.generate(**inputs, use_cache=True, max_new_tokens=2048)
            # outputs_text = tokenizer.batch_decode([outputs[0]], skip_special_tokens=True)
            # end llama 3.1 & cendol inference setup

            # gemma3 setup
            outputs = model.generate(
                **tokenizer([row['text']], return_tensors = "pt").to("cuda"),
                max_new_tokens = 1024, # Increase for longer outputs!
                # Recommended Gemma-3 settings!
                # temperature = 1.0, top_p = 0.95, top_k = 64,
                #qwen3 temp
                temperature = 0.7, top_p = 0.8, top_k = 20
            )
            outputs_text = tokenizer.batch_decode(outputs)
            row['model_answer'] = outputs_text

            temp_df = pd.DataFrame([row])
            inference_results = pd.concat([inference_results, temp_df], ignore_index=True)

            # Save checkpoint periodically
            if (i + 1) % save_frequency == 0:
                save_checkpoint(inference_results, checkpoint_file)
                print(f"\nCheckpoint saved at {i + 1} items")

    except KeyboardInterrupt:
        print("\nInterrupted by user. Saving checkpoint...")
        save_checkpoint(inference_results, checkpoint_file)
        raise
    except Exception as e:
        print(f"\nError occurred: {str(e)}. Saving checkpoint...")
        save_checkpoint(inference_results, checkpoint_file)
        raise

    # Save final checkpoint
    save_checkpoint(inference_results, checkpoint_file)
    print("\nInference completed and final results saved")

    return inference_results

In [ ]:
model.eval()
inference_results = run_inference_with_checkpointing(
    dataset=test,
    model=model,
    tokenizer=tokenizer,
    checkpoint_file=f'/content/drive/MyDrive/model_outputs/inference_result_{model_id.split("/")[1]}-finetuned.csv',
    save_frequency=10
)

Resuming from 0 previously processed items


  9%|▉         | 10/112 [10:40<2:03:19, 72.54s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Checkpoint saved at 10 items


 18%|█▊        | 20/112 [21:01<1:48:14, 70.59s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Checkpoint saved at 20 items


 27%|██▋       | 30/112 [32:18<1:18:07, 57.16s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Checkpoint saved at 30 items


 35%|███▍      | 39/112 [41:25<1:15:54, 62.40s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
